# ATAC-seq Quality Report Analysis

Here we explain the key fields in an ATAC-seq quality report JSON from the ENCODE [atac-seq-pipeline](https://github.com/ENCODE-DCC/atac-seq-pipeline/tree/master). These metrics providing insights into the data quality and characteristics.

## Detailed ATAC-seq Fields

The fields in bold are particularly important for assessing the overall quality of ATAC-seq data.


| Field Path                                            | Explanation                                                   | Good Value Range |
|-------------------------------------------------------|---------------------------------------------------------------|------------------|
| `general.date`                                        | Date of the report generation.                                | N/A              |
| `general.title`                                       | Title or identifier of the ATAC-seq experiment.               | N/A              |
| **`align.dup.pct_duplicate_reads`**                   | Percentage of duplicate reads.                                | `< 10%`          |
| **`align.nodup_samstat.pct_mapped_reads`**            | Percentage of reads mapped to the genome.                     | `> 70%`          |
| **`align.nodup_samstat.pct_properly_paired_reads`**   | Percentage of reads properly paired in sequencing.            | `> 70%`          |
| **`align.frag_len_stat.frac_reads_in_nfr`**           | Fraction of reads in nucleosome-free regions (NFR).           | `~0.4 or higher` |
| `align.frag_len_stat.nfr_over_mono_nuc_reads`         | Ratio of NFR over mononucleosomal reads.                      | `> 2.5`          |
| `align.frag_len_stat.nfr_peak_exists`                 | Indicates existence of NFR peak.                              | `true`           |
| `align.frag_len_stat.mono_nuc_peak_exists`            | Indicates existence of mononucleosomal peak.                  | `true`           |
| `align.frag_len_stat.di_nuc_peak_exists`              | Indicates existence of dinucleosomal peak.                    | `true`           |
| `align.frac_reads_in_annot.fri_dhs`                   | Fraction of reads in DHS sites.                               | N/A              |
| `align.frac_reads_in_annot.fri_blacklist`             | Fraction of reads in blacklist regions.                       | `< 0.01`         |
| `align.frac_reads_in_annot.fri_prom`                  | Fraction of reads in promoter regions.                        | N/A              |
| `align.frac_reads_in_annot.fri_enh`                   | Fraction of reads in enhancer regions.                        | N/A              |
| **`lib_complexity.lib_complexity.NRF`**               | Non-Redundant Fraction, measuring library complexity.         | `> 0.8`          |
| **`lib_complexity.lib_complexity.PBC1`**              | PCR Bottleneck Coefficient 1, indicating PCR duplication.     | `> 0.9`          |
| `lib_complexity.lib_complexity.PBC2`                  | PCR Bottleneck Coefficient 2, another PCR duplication metric. | `> 1.0`          |
| **`replication.num_peaks.rep1.num_peaks`**            | Number of peaks identified.                                   | `> 50000`        |
| `peak_stat.peak_region_size.rep1.min_size`            | Minimum size of the peak regions.                             | `> 150`          |
| `peak_stat.peak_region_size.rep1.mean`                | Mean size of the peak regions.                                | N/A              |
| **`align_enrich.xcor_score.rep1.NSC`**                | Normalized Strand Coefficient, assessing signal-to-noise.     | `> 1.1`          |
| `align_enrich.xcor_score.rep1.RSC`                    | Relative Strand Coefficient, another signal-to-noise metric.  | `> 1.0`          |
| **`peak_enrich.frac_reads_in_peaks.macs2.rep1.frip`** | Fraction of reads in peaks (FRiP) using MACS2 peak caller.    | `> 0.1`          |



In [1]:
import pandas as pd
import json
import os


def find_json_files(parent_folder):
    """
    Recursively finds all JSON files in subfolders of the given parent folder.

    :param parent_folder: Path to the parent folder
    :return: List of paths to JSON files
    """
    json_files = []
    for root, dirs, files in os.walk(parent_folder):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files


def read_json_fields(json_file, fields):
    """Reads specified fields from a JSON file."""
    with open(json_file, 'r') as file:
        data = json.load(file)

        row_data = {}
        for field in fields:
            field_val = data.copy()
            properties = field.split('.')
            for prop in properties:
                if isinstance(field_val, str):
                    field_val = json.loads(field_val)
                field_val = field_val[prop]

            row_data[properties[-1]] = field_val
        return row_data


def create_dataframe(folder_path, fields):
    """Creates a DataFrame from JSON files in the given folder."""
    rows = []
    json_files = find_json_files(folder_path)
    for json_path in json_files:
        row = read_json_fields(json_path, fields)
        rows.append(row)

    return pd.DataFrame(rows)


# List of field paths to extract from the JSON files
fields = [
    'general.date',
    'general.title',
    # 'align.dup.rep1.pct_duplicate_reads',
    # 'align.nodup_samstat.rep1.pct_mapped_reads',
    # 'align.nodup_samstat.rep1.pct_properly_paired_reads',
    'align.frag_len_stat.rep1.frac_reads_in_nfr',
    'align.frag_len_stat.rep1.nfr_over_mono_nuc_reads',
    'align.frag_len_stat.rep1.nfr_peak_exists',
    'align.frag_len_stat.rep1.mono_nuc_peak_exists',
    'align.frag_len_stat.rep1.di_nuc_peak_exists',
    'align.frac_reads_in_annot.rep1.fri_dhs',
    'align.frac_reads_in_annot.rep1.fri_blacklist',
    'align.frac_reads_in_annot.rep1.fri_prom',
    'align.frac_reads_in_annot.rep1.fri_enh',
    'lib_complexity.lib_complexity.rep1.NRF',
    'lib_complexity.lib_complexity.rep1.PBC1',
    'lib_complexity.lib_complexity.rep1.PBC2',
    'replication.num_peaks.rep1.num_peaks',
    # 'peak_stat.peak_region_size.rep1.min_size',
    'peak_stat.peak_region_size.rep1.mean',
    'align_enrich.xcor_score.rep1.NSC',
    'align_enrich.xcor_score.rep1.RSC',
    'peak_enrich.frac_reads_in_peaks.macs2.rep1.frip'
]

folder_path = '../data/'

df = create_dataframe(folder_path, fields)
df

,date,title,frac_reads_in_nfr,nfr_over_mono_nuc_reads,nfr_peak_exists,mono_nuc_peak_exists,di_nuc_peak_exists,fri_dhs,fri_blacklist,fri_prom,fri_enh,NRF,PBC1,PBC2,num_peaks,mean,NSC,RSC,frip
0,2024-01-10 06:09:41,TCGA-AP-A051,0.346145,0.981242,True,True,True,0.432471,0.002295,0.214927,0.302420,0.988663,0.989174,96.613347,299091,698.216677,2.199994,1.862752,0.416858
1,2024-01-10 09:32:24,TCGA-AY-A54L,0.312177,1.022006,True,True,True,0.440085,0.001682,0.193148,0.338942,0.995216,0.995265,212.206257,279590,681.531296,1.877674,1.949106,0.366214
2,2024-01-11 04:24:41,TCGA-DU-6407,0.423323,1.237476,True,True,True,0.453153,0.001252,0.222062,0.330625,0.992130,0.992190,128.591784,229710,653.739567,2.229879,1.624910,0.378840
3,2024-01-10 19:14:44,TCGA-B9-A44B,0.317012,1.129282,True,True,True,0.276370,0.002992,0.101635,0.289806,0.998843,0.998882,909.951737,293561,594.796683,1.202934,2.592615,0.185110
4,2024-01-10 12:26:32,TCGA-B1-A47N,0.401889,1.302718,True,True,True,0.516189,0.002071,0.261557,0.311906,0.988646,0.988672,88.098758,202591,624.825140,2.994427,1.473605,0.447667
5,2024-01-09 13:01:12,TCGA-A6-A567,0.405408,1.215256,True,True,True,0.491502,0.001692,0.236802,0.320220,0.988347,0.988382,86.003628,275256,615.590628,2.502083,1.573176,0.423152
6,2024-01-20 10:46:22,TCGA-IA-A40X,0.392451,1.588995,True,True,True,0.611005,0.001052,0.323416,0.347298,0.975650,0.977003,46.161037,285193,688.836868,3.177248,1.753729,0.596647
7,2024-01-11 08:02:49,TCGA-HE-A5NH,0.482740,1.836700,True,True,True,0.576222,0.001630,0.298424,0.322387,0.980538,0.980713,52.069319,226808,624.423706,3.463414,1.535592,0.527078
8,2024-01-09 23:40:39,TCGA-AA-A01X,0.377609,1.091841,True,True,True,0.524091,0.001284,0.219911,0.390450,0.990223,0.990225,101.985691,253742,634.296794,2.472689,1.601339,0.462303
9,2024-01-10 15:34:47,TCGA-B5-A0JN,0.276540,1.215163,True,True,True,0.386098,0.001770,0.178827,0.317152,0.996945,0.996987,334.891692,298964,611.575594,1.643618,2.134132,0.315661
